# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/ (DOES NOT WORK)
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [8]:
import json
from pandas import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization 

In [9]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [10]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [11]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [12]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [13]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [18]:
wbp = pd.read_json('data/world_bank_projects.json')
wbp.tail()

,sector,supplementprojectflg,projectfinancialtype,prodline,mjtheme,idacommamt,impagency,project_name,mjthemecode,closingdate,...,majorsector_percent,board_approval_month,theme_namecode,countryname,url,source,projectstatusdisplay,ibrdcommamt,sector_namecode,_id
495,"[{'Name': 'General agriculture, fishing and fo...",N,OTHER,RE,"[Financial and private sector development, Soc...",0,THE COMPETITIVENESS COMPANY,Technological Scale Up for Youth-led Urban Orn...,"4,7",2013-08-31T00:00:00Z,...,"[{'Percent': 50, 'Name': 'Agriculture, fishing...",August,"[{'code': '40', 'name': 'Regulation and compet...",Jamaica,http://www.worldbank.org/projects/P127299/tech...,IBRD,Closed,0,"[{'code': 'AZ', 'name': 'General agriculture, ...",{'$oid': '52b213b38594d8a2be17c96f'}
496,[{'Name': 'Central government administration'}...,N,IDA,PE,"[Human development, Trade and integration, Pub...",20000000,MINISTRY OF FINANCE,Lao Eight Poverty Reduction Support,"8,5,2,8",2012-12-31T00:00:00Z,...,"[{'Percent': 36, 'Name': 'Public Administratio...",August,"[{'code': '63', 'name': 'Child health'}, {'cod...",Lao People's Democratic Republic,http://www.worldbank.org/projects/P125298/lao-...,IBRD,Closed,0,"[{'code': 'BC', 'name': 'Central government ad...",{'$oid': '52b213b38594d8a2be17c970'}
497,"[{'Name': 'Irrigation and drainage'}, {'Name':...",N,OTHER,RE,[Rural development],0,MINISTRY OF AGRICULTURE AND PRODUCER ORGANISAT...,Second Emergency Agricultural Productivity Sup...,"10,6",2014-12-31T00:00:00Z,...,"[{'Percent': 40, 'Name': 'Agriculture, fishing...",August,"[{'code': '91', 'name': 'Global food crisis re...",Republic of Guinea,http://www.worldbank.org/projects/P128309/seco...,IBRD,Active,0,"[{'code': 'AI', 'name': 'Irrigation and draina...",{'$oid': '52b213b38594d8a2be17c971'}
498,[{'Name': 'Agricultural extension and research...,N,IBRD,PE,"[Rural development, Rural development, Rural d...",0,INDONESIAN AGENCY FOR AGRICULTURAL RESEARCH AN...,Sustainable Management of Agricultural Researc...,"10,10,10",2017-09-30T00:00:00Z,...,"[{'Percent': 80, 'Name': 'Agriculture, fishing...",August,"[{'code': '78', 'name': 'Rural services and in...",Republic of Indonesia,http://www.worldbank.org/projects/P117243/sust...,IBRD,Active,80000000,"[{'code': 'AB', 'name': 'Agricultural extensio...",{'$oid': '52b213b38594d8a2be17c972'}
499,"[{'Name': 'Urban Transport'}, {'Name': 'Public...",N,IDA,PE,"[Urban development, Human development, Trade a...",300000000,MINISTRY OF ROADS,KENYA: NATIONAL URBAN TRANSPORT IMPROVEMENT PR...,"9,8,5,4",2018-12-31T00:00:00Z,...,"[{'Percent': 79, 'Name': 'Transportation'}, {'...",August,"[{'code': '73', 'name': 'Municipal governance ...",Republic of Kenya,http://www.worldbank.org/projects/P126321/keny...,IBRD,Active,0,"[{'code': 'TC', 'name': 'Urban Transport'}, {'...",{'$oid': '52b213b38594d8a2be17c973'}


In [37]:
# 1. Find the 10 countries with most projects
wbp.groupby('countryname').size().sort_values(ascending=False).iloc[:10]

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
dtype: int64

In [82]:
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
wbp_json = json.load(open('data/world_bank_projects.json'))
mpt = json_normalize(wbp_json, 'mjtheme_namecode')
mpt.groupby(['code', 'name']).size().sort_values(ascending=False).iloc[:10]

code  name                                        
11    Environment and natural resources management    223
10    Rural development                               202
8     Human development                               197
2     Public sector governance                        184
6     Social protection and risk management           158
4     Financial and private sector development        130
7     Social dev/gender/inclusion                     119
5     Trade and integration                            72
9     Urban development                                47
1     Economic management                              33
dtype: int64

In [108]:
# 3. In 2. above you will notice that some entries have only the code and the name is missing. 
#    Create a dataframe with the missing names filled in.
wbp_json = json.load(open('data/world_bank_projects.json'))
mpt = json_normalize(wbp_json, 'mjtheme_namecode')

code_dict = {'1':'Economic management', '2':'Public sector governance', 
            '3':'Rule of law', '4':'Financial and private sector development',
            '5':'Trade and integration', '6':'Social protection and risk management',
            '7':'Social dev/gender/inclusion', '8':'Human development',
            '9':'Urban development', '10':'Rural development',
            '11':'Environment and natural resources management'}

def code_map(c):
    return code_dict[c];

mpt['name'] = [code_map(c) for c in mpt['code']]
mpt.groupby(['code', 'name']).size().sort_values(ascending=False).iloc[:10]

code  name                                        
11    Environment and natural resources management    250
10    Rural development                               216
8     Human development                               210
2     Public sector governance                        199
6     Social protection and risk management           168
4     Financial and private sector development        146
7     Social dev/gender/inclusion                     130
5     Trade and integration                            77
9     Urban development                                50
1     Economic management                              38
dtype: int64